# 05tools_1: Prediction
Predictions from models created in the 05 series of notebooks.

This notebook is a collection of examples that showcase many ways to serve models:
- Online:
    - Vertex AI Endpoints: Python, REST, CLI (gcloud)
    - Local with TensorFlow ModelServer
    - Remote with Cloud Run with TensorFlow ModelServer
- Batch:
    - BigQuery ML Model Import
    - Vertex AI Batch Prediction Jobs

TODO:
 - [ ] Demonstrate endpoint scaling
     - [ ] Vertex AI Endpoint
     - [ ] Cloud Run Endpoint
- [IP] Batch Predictions with Vertex AI
- [ ] Managing Multiple Models On Endpoints
    - [ ] Traffic Split
    - [ ] Deployment Pools
    - [ ] Custom Endpoints
- [ ] Cloud Functions Serving

---
## Setup

inputs:

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [770]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '05tools_1'
SERIES = '05'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [781]:
from google.cloud import aiplatform
from google.cloud import bigquery

import tensorflow as tf

from datetime import datetime
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

import asyncio
import time
import multiprocessing

clients:

In [338]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()

parameters:

In [339]:
BUCKET = PROJECT_ID
DIR = f"temp/{NOTEBOOK}"

In [1108]:
# Give service account roles/storage.objectAdmin permissions
# Console > IAM > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:

In [340]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get Endpoint

[Endpoint Properties and Methods](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Endpoint):

```python
endpoint
endpoint.display_name
endpoint.resource_name
endpoint.traffic_split
endpoint.list_models()
```

In [341]:
endpoints = aiplatform.Endpoint.list(filter = f"display_name={SERIES}_{DATANAME}")
endpoint = endpoints[0]

---
## Retrieve Records For Prediction

In [342]:
n = 1000
pred = bq.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT {n}").to_dataframe()

In [343]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,32799,1.153477,-0.047859,1.358363,1.480620,-1.222598,-0.481690,-0.654461,0.128115,0.907095,...,-0.025964,0.701843,0.417245,-0.257691,0.060115,0.035332,0.0,0,e9d16028-4b41-4753-87ee-041d33642ae9,TEST
1,35483,1.286640,0.072917,0.212182,-0.269732,-0.283961,-0.663306,-0.016385,-0.120297,-0.135962,...,0.052674,0.076792,0.209208,0.847617,-0.086559,-0.008262,0.0,0,8b319d3a-2b2d-445b-a9a2-0da3d664ec2a,TEST
2,163935,1.961967,-0.247295,-1.751841,-0.268689,0.956431,0.707211,0.020675,0.189433,0.455055,...,0.186420,-1.621368,-0.131098,0.034276,-0.004909,-0.090859,0.0,0,788afb87-60aa-4482-8b48-c924bec634aa,TEST
3,30707,-0.964364,0.176372,2.464128,2.672539,0.145676,-0.152913,-0.591983,0.305066,-0.148034,...,-0.024200,0.365226,-0.745369,-0.060544,0.095692,0.217639,0.0,0,473d0936-1974-4ae8-ab70-230e7599bd3f,TEST


Remove columns not included as features in the model:

In [344]:
newobs = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')
#newobs[0]

In [345]:
len(newobs)

1000

---
## Online Predictions: Methods for Vertex AI Endpoints

### Get Prediction: Python Client

In [879]:
instances = [json_format.ParseDict(newobs[0], Value())]

prediction = endpoint.predict(instances=instances)
prediction

Prediction(predictions=[[0.999275744, 0.000724321057]], deployed_model_id='7077164921854623744', model_version_id='', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05i_fraud', explanations=None)

In [880]:
prediction.predictions[0]

[0.999275744, 0.000724321057]

In [881]:
np.argmax(prediction.predictions[0])

0

### Get Prediction: REST

#### Method 1: Command Line CURL

In [349]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newobs[0]]}))

In [350]:
prediction = !curl -s POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

prediction = json.loads(''.join([p.strip() for p in prediction]))
prediction

{'predictions': [[0.999275744, 0.000724321057]],
 'deployedModelId': '7077164921854623744',
 'model': 'projects/1026793852137/locations/us-central1/models/model_05i_fraud',
 'modelDisplayName': '05i_fraud'}

In [351]:
prediction['predictions'][0]

[0.999275744, 0.000724321057]

In [352]:
np.argmax(prediction['predictions'][0])

0

#### Method 2: Python with requests

In [429]:
import requests

In [435]:
token = !gcloud auth application-default print-access-token
headers = {"content-type": "application/json; charset=utf-8", "Authorization": f'Bearer {token[0]}'}
json_response = requests.post(f'https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict', data=json.dumps({"instances": [newobs[0]]}), headers=headers)

In [436]:
print(json_response.text)

{
  "predictions": [
    [
      0.999275744,
      0.000724321057
    ]
  ],
  "deployedModelId": "7077164921854623744",
  "model": "projects/1026793852137/locations/us-central1/models/model_05i_fraud",
  "modelDisplayName": "05i_fraud"
}



In [437]:
predictions = json.loads(json_response.text)['predictions']
predictions

[[0.999275744, 0.000724321057]]

In [438]:
np.argmax(predictions[0])

0

### Get Prediction: gcloud (CLI)

In [353]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newobs[0]]}))

In [354]:
prediction = !gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json
prediction

['Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]',
 '[[0.999275744, 0.000724321057]]']

In [355]:
import ast
prediction = ast.literal_eval(prediction[1])
prediction[0]

[0.999275744, 0.000724321057]

In [356]:
np.argmax(prediction[0])

0

---
## Online Predictions: Synchronous Examples

In [358]:
predictions = []
start = time.perf_counter()
for p in range(len(newobs)):
    instances = [json_format.ParseDict(newobs[p], Value())]
    prediction = endpoint.predict(instances=instances)
    predictions.append(np.argmax(prediction.predictions[0]))
elapsed = time.perf_counter() - start
print(f'{elapsed:0.5f} seconds')

12.83824 seconds


In [359]:
from collections import Counter
c = Counter(predictions)
c

Counter({0: 997, 1: 3})

In [360]:
[i for i, j in enumerate(predictions) if j == 1]

[85, 251, 473]

In [361]:
pred.iloc[[i for i, j in enumerate(predictions) if j == 1]]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
85,148074,-2.219219,0.727831,-5.458230,5.924850,3.932464,-3.085984,-1.677870,0.865075,-3.177260,...,-0.028752,0.025723,-0.825835,-0.013089,0.413291,-0.131387,0.0,1,a13daf1c-d562-4f9e-b3a4-ba0bfe58d6fa,TEST
251,129668,0.753356,2.284988,-5.164492,3.831112,-0.073622,-1.316596,-1.855495,0.831079,-1.567514,...,0.187697,0.358433,-0.488934,-0.258802,0.296145,-0.047174,2.0,1,ccbcb7b2-79be-4ad0-9b75-771f695ef69b,TEST
473,56887,-0.075483,1.812355,-2.566981,4.127549,-1.628532,-0.805895,-3.390135,1.019353,-2.451251,...,-0.143624,0.013566,0.634203,0.213693,0.773625,0.387434,5.0,1,ba8d853c-0e20-4422-a9f0-03daa3b492e7,TEST


---
## Online Predictions: Asynchronous Examples
Using Multiprocessing

In [362]:
from google.cloud import aiplatform_v1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
parent = f"projects/{PROJECT_ID}/locations/{REGION}"

client = aiplatform_v1.PredictionServiceAsyncClient(client_options = client_options)

In [363]:
endpoint.resource_name

'projects/1026793852137/locations/us-central1/endpoints/7252545822577917952'

In [364]:
await client.predict(endpoint = endpoint.resource_name, instances = instances)

predictions {
  list_value {
    values {
      number_value: 0.999950767
    }
    values {
      number_value: 4.92256549e-05
    }
  }
}
deployed_model_id: "7077164921854623744"
model: "projects/1026793852137/locations/us-central1/models/model_05i_fraud"
model_display_name: "05i_fraud"

In [391]:
limit = asyncio.Semaphore(100)

predictions = [None] * len(newobs)

async def predictor(p, newob):
    instances = [json_format.ParseDict(newob, Value())]
    async with limit:
        prediction = await client.predict(endpoint = endpoint.resource_name, instances=instances)
        if limit.locked():
            await asyncio.sleep(.01)
            
    predictions[p] = np.argmax(prediction.predictions[0])
    
async def runner(newobs):
    tasks = []
    for p in range(len(newobs)):
        task = asyncio.create_task(predictor(p, newobs[p]))
        tasks.append(task)
        
    results = await asyncio.gather(*tasks)

start = time.perf_counter()
await runner(newobs)
elapsed = time.perf_counter() - start
print(f'{elapsed:0.5f} seconds')

1.26097 seconds


In [392]:
len(predictions)

1000

In [393]:
from collections import Counter
c = Counter(predictions)
c

Counter({0: 997, 1: 3})

In [394]:
[i for i, j in enumerate(predictions) if j == 1]

[85, 251, 473]

In [395]:
pred.iloc[[i for i, j in enumerate(predictions) if j == 1]]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
85,148074,-2.219219,0.727831,-5.458230,5.924850,3.932464,-3.085984,-1.677870,0.865075,-3.177260,...,-0.028752,0.025723,-0.825835,-0.013089,0.413291,-0.131387,0.0,1,a13daf1c-d562-4f9e-b3a4-ba0bfe58d6fa,TEST
251,129668,0.753356,2.284988,-5.164492,3.831112,-0.073622,-1.316596,-1.855495,0.831079,-1.567514,...,0.187697,0.358433,-0.488934,-0.258802,0.296145,-0.047174,2.0,1,ccbcb7b2-79be-4ad0-9b75-771f695ef69b,TEST
473,56887,-0.075483,1.812355,-2.566981,4.127549,-1.628532,-0.805895,-3.390135,1.019353,-2.451251,...,-0.143624,0.013566,0.634203,0.213693,0.773625,0.387434,5.0,1,ba8d853c-0e20-4422-a9f0-03daa3b492e7,TEST


---
## Vertex AI Batch Prediction Jobs

Create a Vertex AI Batch Predictions Job in one of these ways:
- Directly with [aiplatform.BatchPredictionJob.create()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.BatchPredictionJob)
- From a Model with [aiplatform.Model.batch_predict()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_batch_predict)

### Model Information
Using the model on the endpoint selected at the top of this notebook:

In [759]:
endpoint.list_models()[0]

id: "7077164921854623744"
model: "projects/1026793852137/locations/us-central1/models/model_05i_fraud"
display_name: "05i_fraud"
create_time {
  seconds: 1659091859
  nanos: 91291000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
model_version_id: "1"

In [766]:
model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model+f'@{endpoint.list_models()[0].model_version_id}'
)

In [761]:
model.display_name

'05i_fraud'

In [762]:
model.resource_name

'projects/1026793852137/locations/us-central1/models/model_05i_fraud'

In [763]:
model.version_id

'1'

In [764]:
model.version_description

'time-20220728003419'

In [765]:
model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/model_05i_fraud@1'

### Method 1: With aiplatform.BatchPredictionJob.create()

In [773]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
batchJob = aiplatform.BatchPredictionJob.create(
    job_display_name = f'{SERIES}_{DATANAME}_{TIMESTAMP}',
    model_name = model.versioned_resource_name,
    bigquery_source= f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}_prepped',
    bigquery_destination_prefix= f'bq://{PROJECT_ID}.{DATANAME}',
    machine_type = DEPLOY_COMPUTE,
    accelerator_count = 0,
    starting_replica_count = 1,
    max_replica_count = 2,
    batch_size = 100,
    sync = False #if True the call will wait for the job to complete
)

Creating BatchPredictionJob


INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob


BatchPredictionJob created. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504


To use this BatchPredictionJob in another session:


...

INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_RUNNING


BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_RUNNING


BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_RUNNING


In [775]:
batchJob.wait()

BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_RUNNING


BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_RUNNING


BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_RUNNING


...

INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_RUNNING


BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_SUCCEEDED


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504 current state:
JobState.JOB_STATE_SUCCEEDED


BatchPredictionJob run completed. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob run completed. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/2697443146417045504


### Method 2: With aiplatform.Model.batch_predict()

In [776]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
batchJob = model.batch_predict(
    job_display_name = f'{SERIES}_{DATANAME}_{TIMESTAMP}',
    bigquery_source= f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}_prepped',
    instances_format = "bigquery",
    bigquery_destination_prefix= f'bq://{PROJECT_ID}.{DATANAME}',
    predictions_format = "bigquery",
    machine_type = DEPLOY_COMPUTE,
    accelerator_count = 0,
    starting_replica_count = 1,
    max_replica_count = 10,
    batch_size = 100,
    sync = False #if True the call will wait for the job to complete
)

Creating BatchPredictionJob


INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob


BatchPredictionJob created. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712


To use this BatchPredictionJob in another session:


...

INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/8778850755739123712?project=1026793852137


BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_RUNNING


BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_RUNNING


In [777]:
batchJob.wait()

BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_RUNNING


BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_RUNNING


BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_RUNNING


...

INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_RUNNING


BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_SUCCEEDED


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712 current state:
JobState.JOB_STATE_SUCCEEDED


BatchPredictionJob run completed. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob run completed. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/8778850755739123712


---
## Batch Predictions: BigQuery ML

Load a model to BigQuery and use BQML to create predictions.  Need the URI of TensorFlow model.

### Model Information
Using the model on the endpoint selected at the top of this notebook:

In [759]:
endpoint.list_models()[0]

id: "7077164921854623744"
model: "projects/1026793852137/locations/us-central1/models/model_05i_fraud"
display_name: "05i_fraud"
create_time {
  seconds: 1659091859
  nanos: 91291000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
model_version_id: "1"

In [766]:
model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model+f'@{endpoint.list_models()[0].model_version_id}'
)

In [761]:
model.display_name

'05i_fraud'

In [762]:
model.resource_name

'projects/1026793852137/locations/us-central1/models/model_05i_fraud'

In [763]:
model.version_id

'1'

In [764]:
model.version_description

'time-20220728003419'

In [767]:
model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/model_05i_fraud@1'

In [768]:
model.uri

'gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model'

In [769]:
model.name

'model_05i_fraud'

### Import Model Into BigQuery

In [122]:
query = f'''
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATANAME}.{model.name}`
    OPTIONS(
        MODEL_TYPE = 'TENSORFLOW',
        MODEL_PATH = '{model.uri}/*')
'''

In [123]:
print(query)


CREATE OR REPLACE MODEL `statmike-mlops-349915.fraud.model_05i_fraud`
    OPTIONS(
        MODEL_TYPE = 'TENSORFLOW',
        MODEL_PATH = 'gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model/*')



In [124]:
job = bq.query(query = query)
job.result()
(job.ended-job.started).total_seconds()

3.605

### Generate Predictions With BigQuery (ML.Predict)

In [125]:
query = f'''
SELECT *
FROM ML.PREDICT(
    MODEL `{PROJECT_ID}.{DATANAME}.{model.name}`, (
        SELECT * 
        FROM {DATANAME}.{DATANAME}_prepped
        WHERE splits='TEST' AND Class = 1
        LIMIT 10
    )
)
'''
results = bq.query(query = query).to_dataframe()
results

,dense,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,"[0.0016647283919155598, 0.9983352422714233]",148074,-2.219219,0.727831,-5.458230,5.924850,3.932464,-3.085984,-1.677870,0.865075,...,-0.028752,0.025723,-0.825835,-0.013089,0.413291,-0.131387,0.0,1,a13daf1c-d562-4f9e-b3a4-ba0bfe58d6fa,TEST
1,"[0.016800928860902786, 0.9831990003585815]",129668,0.753356,2.284988,-5.164492,3.831112,-0.073622,-1.316596,-1.855495,0.831079,...,0.187697,0.358433,-0.488934,-0.258802,0.296145,-0.047174,2.0,1,ccbcb7b2-79be-4ad0-9b75-771f695ef69b,TEST
2,"[0.0005459527601487935, 0.9994540810585022]",56887,-0.075483,1.812355,-2.566981,4.127549,-1.628532,-0.805895,-3.390135,1.019353,...,-0.143624,0.013566,0.634203,0.213693,0.773625,0.387434,5.0,1,ba8d853c-0e20-4422-a9f0-03daa3b492e7,TEST
3,"[0.8952898979187012, 0.10471007227897644]",146998,-2.064240,2.629739,-0.748406,0.694992,0.418178,1.392520,-1.697801,-6.333065,...,0.459861,-1.051685,0.209178,-0.319859,0.015434,-0.050117,8.0,1,f390e04f-14f4-4c77-b18d-6adb8da79b54,TEST
4,"[0.11997880041599274, 0.8800212740898132]",78725,-4.312479,1.886476,-2.338634,-0.475243,-1.185444,-2.112079,-2.122793,0.272565,...,-1.114692,0.269069,-0.020572,-0.963489,-0.918888,0.001454,60.0,1,37db7004-4459-48a3-8464-24c5ff19cc3b,TEST
5,"[4.79785031131931e-16, 0.9999999403953552]",14152,-4.710529,8.636214,-15.496222,10.313349,-4.351341,-3.322689,-10.788373,5.060381,...,0.554408,-1.204042,-0.450685,0.641836,1.605958,0.721644,1.0,1,5bb6c913-1486-4421-a791-f13511b24f1f,TEST
6,"[6.733386092618032e-14, 0.9999999403953552]",21419,-17.467710,10.114816,-24.202142,11.805469,-10.198046,-2.579938,-17.656788,2.256902,...,1.296817,-1.055104,0.111792,0.679695,2.093541,-1.425491,1.0,1,8922139c-c818-4d35-a996-7dc72d19216a,TEST
7,"[1.3924342056270689e-05, 0.9999859929084778]",143456,-2.006582,3.676577,-5.463811,7.232058,-1.627859,-0.996755,-4.299833,2.268867,...,0.167947,-0.449864,0.023702,0.536905,0.485864,-0.042393,1.0,1,ebd4dc94-3c92-41c8-b158-88ea102a5c23,TEST
8,"[1.3694696463062428e-05, 0.9999862909317017]",40662,-4.446847,-0.014793,-5.126307,6.945130,5.269255,-4.297177,-2.591242,0.342671,...,-0.226017,-0.401236,0.856124,0.661272,0.492560,0.971834,1.0,1,632895ce-64c7-4296-a638-ca415be392df,TEST
9,"[0.9690614938735962, 0.030938537791371346]",77171,1.118560,1.291858,-1.298805,2.135772,0.772204,-1.147291,0.390578,-0.107072,...,-0.102326,0.017911,0.650302,-0.332366,0.105949,0.128124,1.0,1,b17e46f7-9352-4bd0-91d4-4260be816363,TEST


---
## Local Predictions: With TensorFlow ModelServer
Locally run [TensorFlow Serving with Docker](https://www.tensorflow.org/tfx/serving/docker#serving_example)

### Model Information
Using the model on the endpoint selected at the top of this notebook:

In [759]:
endpoint.list_models()[0]

id: "7077164921854623744"
model: "projects/1026793852137/locations/us-central1/models/model_05i_fraud"
display_name: "05i_fraud"
create_time {
  seconds: 1659091859
  nanos: 91291000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
model_version_id: "1"

In [766]:
model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model+f'@{endpoint.list_models()[0].model_version_id}'
)

In [761]:
model.display_name

'05i_fraud'

In [762]:
model.resource_name

'projects/1026793852137/locations/us-central1/models/model_05i_fraud'

In [763]:
model.version_id

'1'

In [764]:
model.version_description

'time-20220728003419'

In [767]:
model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/model_05i_fraud@1'

In [768]:
model.uri

'gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model'

In [769]:
model.name

'model_05i_fraud'

Locate the model files:

In [778]:
model.uri

'gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model'

Review the local directory for this notebook (created above):

In [779]:
DIR

'temp/05tools_1'

In [780]:
!ls {DIR}

model  request.json


Copy the model files to the local directory for this notebook:

In [711]:
!gsutil cp -R {model.uri} {DIR}

Copying gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model/saved_model.pb...
/ [1 files][515.8 KiB/515.8 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model/variables/variables.data-00000-of-00001...
Copying gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model/variables/variables.index...
/ [3 files][526.5 KiB/526.5 KiB]                                                
Operation completed over 3 objects/526.5 KiB.                                    


In [712]:
!ls {DIR}

model  request.json


In [713]:
!ls {DIR}/model

saved_model.pb	variables


### Load the Model and Review

In [782]:
reloaded_model = tf.saved_model.load(f'{DIR}/model')

In [783]:
reloaded_model.signatures

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(Amount, Time, V1, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V2, V20, V21, V22, V23, V24, V25, V26, V27, V28, V3, V4, V5, V6, V7, V8, V9) at 0x7F6EA922DF10>})

In [784]:
reloaded_model.signatures['serving_default']

<ConcreteFunction signature_wrapper(Amount, Time, V1, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V2, V20, V21, V22, V23, V24, V25, V26, V27, V28, V3, V4, V5, V6, V7, V8, V9) at 0x7F6EA922DF10>

In [785]:
reloaded_model.signatures['serving_default'].structured_input_signature

((),
 {'V15': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V15'),
  'Time': TensorSpec(shape=(None, 1), dtype=tf.float32, name='Time'),
  'Amount': TensorSpec(shape=(None, 1), dtype=tf.float32, name='Amount'),
  'V24': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V24'),
  'V19': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V19'),
  'V23': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V23'),
  'V27': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V27'),
  'V20': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V20'),
  'V5': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V5'),
  'V6': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V6'),
  'V9': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V9'),
  'V28': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V28'),
  'V13': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V13'),
  'V18': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V18'),
  'V11': TensorSpec(shape=(None, 1), dtyp

In [786]:
#!saved_model_cli show --dir {DIR}/model --all

### Download Docker Image and Start Serving Container

In [989]:
!docker pull tensorflow/serving

Using default tag: latest
latest: Pulling from tensorflow/serving
Digest: sha256:6651f4839e1124dbde75ee531825112af0a6b8ef082c88ab14ca53eb69a2e4bb
Status: Image is up to date for tensorflow/serving:latest
docker.io/tensorflow/serving:latest


In [1014]:
command = f'''docker run -t -p 8501:8501 \
-v "/$(pwd)/{DIR}/model:/models/fraud/1" \
-e MODEL_NAME={DATANAME} \
tensorflow/serving'''
print(command)

docker run -t -p 8501:8501 -v "/$(pwd)/temp/05tools_1/model:/models/fraud/1" -e MODEL_NAME=fraud tensorflow/serving


**Run the command above in a subprocess at the local folder of this notebook - use multiprocess.Process():**

In [991]:
!pwd

/home/jupyter/vertex-ai-mlops


In [992]:
import multiprocessing

def docker_runner():
    !{command}
    #!docker run -t -p 8501:8501 -v "/$(pwd)/temp/05tools_1/model:/models/fraud/1" -e MODEL_NAME=fraud tensorflow/serving

def main():
    p = multiprocessing.Process(target=docker_runner)
    p.start()
    return p
    
p = main()

2022-08-03 19:44:33.637600: I tensorflow_serving/model_servers/server.cc:89] Building single TensorFlow model file config:  model_name: fraud model_base_path: /models/fraud
2022-08-03 19:44:33.637949: I tensorflow_serving/model_servers/server_core.cc:465] Adding/updating models.
2022-08-03 19:44:33.637988: I tensorflow_serving/model_servers/server_core.cc:591]  (Re-)adding model: fraud
2022-08-03 19:44:33.738900: I tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: fraud version: 1}
2022-08-03 19:44:33.738946: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: fraud version: 1}
2022-08-03 19:44:33.738966: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: fraud version: 1}
2022-08-03 19:44:33.739028: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:38] Reading SavedModel from: /models/fraud/1
2022-08-03 19:44:33.752621: I external/org_tensorflow/tensorflo

### Get Predictions on Exposed Port

In [993]:
import requests

In [994]:
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fraud:predict', data=json.dumps({"instances": [newobs[0]]}), headers=headers)

In [995]:
print(json_response.text)

{
    "predictions": [[0.999275744, 0.000724321057]
    ]
}


In [996]:
predictions = json.loads(json_response.text)['predictions']
predictions

[[0.999275744, 0.000724321057]]

In [997]:
np.argmax(predictions[0])

0

### Shutdown TensorFlow Serving Container
There are two entities running: a subprocess called `p` and a docker container that was run by the subprocess.  It is not enough to just stop `p` but it might be enough to stop the container and then the subprocess will terminate due to completion.  The command below stop the subprocess `p` and then stop and remove the container.

In [998]:
p.terminate()

In [999]:
p.is_alive()

False

In [1000]:
docker = !docker ps -a
docker

['CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS         PORTS                              NAMES',
 'fbc1bbd07315   tensorflow/serving             "/usr/bin/tf_serving…"   10 seconds ago   Up 8 seconds   8500/tcp, 0.0.0.0:8501->8501/tcp   heuristic_kalam',
 'c5597c5a330a   gcr.io/inverting-proxy/agent   "/bin/sh -c \'/opt/bi…"   8 days ago       Up 8 days                                         proxy-agent']

In [1001]:
for d in docker:
    if 'tensorflow/serving' in d:
        print(d.split()[-1])
        !docker stop {d.split()[-1]}
        !docker rm {d.split()[0]}

heuristic_kalam
heuristic_kalam
fbc1bbd07315


In [1002]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED      STATUS      PORTS     NAMES
c5597c5a330a   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   8 days ago   Up 8 days             proxy-agent


---
## Serving With Cloud Run: TensorFlow ModelServer

### Model Information
Using the model on the endpoint selected at the top of this notebook:

In [1194]:
endpoint.list_models()[0]

id: "7077164921854623744"
model: "projects/1026793852137/locations/us-central1/models/model_05i_fraud"
display_name: "05i_fraud"
create_time {
  seconds: 1659091859
  nanos: 91291000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
model_version_id: "1"

In [1195]:
model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model+f'@{endpoint.list_models()[0].model_version_id}'
)

In [1196]:
model.display_name

'05i_fraud'

In [1197]:
model.resource_name

'projects/1026793852137/locations/us-central1/models/model_05i_fraud'

In [1198]:
model.version_id

'1'

In [1199]:
model.version_description

'time-20220728003419'

In [1200]:
model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/model_05i_fraud@1'

In [1201]:
model.uri

'gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model'

In [1202]:
model.name

'model_05i_fraud'

Locate the model files:

In [1203]:
model.uri

'gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model'

Review the local directory for this notebook (created above):

In [1204]:
DIR

'temp/05tools_1'

In [1205]:
!ls {DIR}

model  request.json


Copy the model files to the local directory for this notebook:

In [1206]:
!gsutil cp -R {model.uri} {DIR}

Copying gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model/saved_model.pb...
/ [1 files][515.8 KiB/515.8 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model/variables/variables.data-00000-of-00001...
Copying gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model/variables/variables.index...
/ [3 files][526.5 KiB/526.5 KiB]                                                
Operation completed over 3 objects/526.5 KiB.                                    


In [1207]:
!ls {DIR}

model  request.json


In [1208]:
!ls {DIR}/model

Dockerfile  saved_model.pb  variables


### Load the Model (local) and Review

In [1209]:
reloaded_model = tf.saved_model.load(f'{DIR}/model')

In [1210]:
reloaded_model.signatures

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(Amount, Time, V1, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V2, V20, V21, V22, V23, V24, V25, V26, V27, V28, V3, V4, V5, V6, V7, V8, V9) at 0x7F6EA9B76F90>})

In [1211]:
reloaded_model.signatures['serving_default']

<ConcreteFunction signature_wrapper(Amount, Time, V1, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V2, V20, V21, V22, V23, V24, V25, V26, V27, V28, V3, V4, V5, V6, V7, V8, V9) at 0x7F6EA9B76F90>

In [1212]:
reloaded_model.signatures['serving_default'].structured_input_signature

((),
 {'V20': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V20'),
  'V3': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V3'),
  'V26': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V26'),
  'V17': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V17'),
  'V27': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V27'),
  'V10': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V10'),
  'V13': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V13'),
  'V12': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V12'),
  'V16': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V16'),
  'V14': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V14'),
  'V4': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V4'),
  'V6': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V6'),
  'V24': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V24'),
  'V22': TensorSpec(shape=(None, 1), dtype=tf.float32, name='V22'),
  'V9': TensorSpec(shape=(None, 1), dtype=tf.floa

In [1213]:
#!saved_model_cli show --dir {DIR}/model --all

### Build Docker Container
This build is local to the notebook.  It could be done on a service like Cloud Build.

In [1214]:
dockerfile = f"""
FROM tensorflow/serving
#ENTRYPOINT [“/usr/bin/env”]
ENV MODEL_NAME={DATANAME}
ENV PORT=8501
COPY . /models/{DATANAME}/1
#RUN ls -la /models/{DATANAME}
CMD tensorflow_model_server --port8500 --rest_api_port=$PORT --model_base_path=/models/{DATANAME} --model_name=$MODEL_NAME
"""
with open(f'{DIR}/model/Dockerfile', 'w') as f:
    f.write(dockerfile)

Create an Image Tag for Artifact Registry - the repository name:

In [1215]:
IMAGE_URI=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PROJECT_ID}/{NOTEBOOK}:latest"
IMAGE_URI

'us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/05tools_1:latest'

Docker build - local:

In [1216]:
!docker build -t $IMAGE_URI {DIR}/model/.

Sending build context to Docker daemon  543.7kB
Step 1/5 : FROM tensorflow/serving
 ---> e874bf5e4700
Step 2/5 : ENV MODEL_NAME=fraud
 ---> Using cache
 ---> a28393663f8b
Step 3/5 : ENV PORT=8501
 ---> Using cache
 ---> ce296056f460
Step 4/5 : COPY . /models/fraud/1
 ---> Using cache
 ---> a9ac3bcc8309
Step 5/5 : CMD tensorflow_model_server --port8500 --rest_api_port=$PORT --model_base_path=/models/fraud --model_name=$MODEL_NAME
 ---> Using cache
 ---> 9fcec03b1da0
Successfully built 9fcec03b1da0
Successfully tagged us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/05tools_1:latest


### Test Docker Container Locally (in a subprocess)
Use the `-rm` flag to indicate the container should be automatically removed once stopped.

In [1217]:
import multiprocessing

def docker_runner():
    !docker run -t --rm -i -p 8501:8501 $IMAGE_URI

def main():
    p = multiprocessing.Process(target=docker_runner)
    p.start()
    return p
    
p = main()

unknown argument: /bin/sh
usage: tensorflow_model_server
Flags:
	--port=8500                      	int32	TCP port to listen on for gRPC/HTTP API. Disabled if port set to zero.
	--grpc_socket_path=""            	string	If non-empty, listen to a UNIX socket for gRPC API on the given path. Can be either relative or absolute path.
	--rest_api_port=0                	int32	Port to listen on for HTTP/REST API. If set to zero HTTP/REST API will not be exported. This port must be different than the one specified in --port.
	--rest_api_num_threads=16        	int32	Number of threads for HTTP/REST API processing. If not set, will be auto set based on number of CPUs.
	--rest_api_timeout_in_ms=30000   	int32	Timeout for HTTP/REST API calls.
	--rest_api_enable_cors_support=false	bool	Enable CORS headers in response
	--enable_batching=false          	bool	enable batching
	--allow_version_labels_for_unavailable_models=false	bool	If true, allows assigning unused version labels to models that are not ava

#### Get Predictions on Exposed Port

In [1218]:
import requests

In [1219]:
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fraud:predict', data=json.dumps({"instances": [newobs[0]]}), headers=headers)

In [1220]:
print(json_response.text)

{
    "predictions": [[0.999275744, 0.000724321057]
    ]
}


In [1221]:
predictions = json.loads(json_response.text)['predictions']
predictions

[[0.999275744, 0.000724321057]]

In [1222]:
np.argmax(predictions[0])

0

#### Shutdown TensorFlow Serving Container
There are two entities running: a subprocess called `p` and a docker container that was run by the subprocess.  It is not enough to just stop `p` but it might be enough to stop the container and then the subprocess will terminate due to completion.  The commands below stop the subprocess `p` and then stop and remove (automatic since run with `-rm` flag) the container.

In [1223]:
p.terminate()

In [1224]:
p.is_alive()

False

In [1225]:
docker = !docker ps -a
docker

['CONTAINER ID   IMAGE                                                                                     COMMAND                  CREATED          STATUS         PORTS                              NAMES',
 'cdaf92c2f3ed   us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/05tools_1:latest   "/usr/bin/tf_serving…"   10 seconds ago   Up 9 seconds   8500/tcp, 0.0.0.0:8501->8501/tcp   jovial_snyder',
 'c5597c5a330a   gcr.io/inverting-proxy/agent                                                              "/bin/sh -c \'/opt/bi…"   9 days ago       Up 9 days                                         proxy-agent']

In [1226]:
for d in docker:
    if f'{IMAGE_URI}' in d:
        print(d.split()[-1])
        !docker stop {d.split()[-1]}

jovial_snyder
jovial_snyder


In [1227]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED      STATUS      PORTS     NAMES
c5597c5a330a   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   9 days ago   Up 9 days             proxy-agent


### Push the Docker Container to Artifact Registry

#### Enable Artifact Registry API:
Check to see if the api is enabled, if not then enable it:

In [1228]:
services = !gcloud services list --format="json" --available --filter=name:artifactregistry.googleapis.com
services = json.loads("".join(services))

if (services[0]['config']['name'] == 'artifactregistry.googleapis.com') & (services[0]['state'] == 'ENABLED'):
    print(f"Artifact Registry is Enabled for This Project: {PROJECT_ID}")
else:
    print(f"Enabeling Artifact Registry for this Project: {PROJECT_ID}")
    !gcloud services enable artifactregistry.googleapis.com

Artifact Registry is Enabled for This Project: statmike-mlops-349915


#### Create A Repository
Check to see if the registry is already created, if not then create it

In [1229]:
check_for_repo = !gcloud artifacts repositories describe {PROJECT_ID} --location={REGION}

if check_for_repo[0].startswith('ERROR'):
    print(f'Creating a repository named {PROJECT_ID}')
    !gcloud  artifacts repositories create {PROJECT_ID} --repository-format=docker --location={REGION} --description="Vertex AI Training Custom Containers"
else:
    print(f'There is already a repository named {PROJECT_ID}')

There is already a repository named statmike-mlops-349915


#### Configure Local Docker to Use GCLOUD CLI

In [1230]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


#### Push The Container to The Repository

In [1231]:
!docker push $IMAGE_URI

The push refers to repository [us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/05tools_1]

808eb166: Preparing 
23850a27: Preparing 
a33781cd: Preparing 
89523b17: Preparing 
f28d5f3c: Preparing 
c6d2db45: Preparing 
bacb0351: Preparing 
e1acaabc: Layer already exists latest: digest: sha256:ad1959dd66e3579cf1aadb256e8d3ac0d712b7e0371c235f520a62753f7421b6 size: 1989


### Deploy as Cloud Run Service
This demonstration creates an open service allowing all traffic.  Review documentation for [Cloud Run](https://cloud.google.com/run/docs/overview/what-is-cloud-run) and the [CLOUD SKD CLI sections](https://cloud.google.com/sdk/gcloud/reference/run) for `gcloud run`.


If you have a policy inforced for 'Domain Restricted Sharing' then it may need adjusting for the project to allow this.  This should be done with care and you may wish to only accept authenticated or internal traffic.  Review options for authentication [here](https://cloud.google.com/run/docs/authenticating/overview).

Updated Org Policy:
- Logged in as Admin
- IAM > Organization Policies
    - Changed to Project (not org level)
    - Filter 'Domain Restricted Sharing'
    - Select and Edit
        - Applies to = Customize
        - Policy enforcement = Replace
        - Rules = Allow all
    - Save

View the Cloud Run Console for this project:

In [1233]:
print(f'https://console.cloud.google.com/run?project={PROJECT_ID}')

https://console.cloud.google.com/run?project=statmike-mlops-349915


In [1234]:
!gcloud run deploy endpoint-$SERIES-$DATANAME --image=$IMAGE_URI --port=8501 --region=$REGION --platform=managed --allow-unauthenticated --no-user-output-enabled

Deploying new service...                                                       
  . Creating Revision...                                                       
  . Routing traffic...                                                         
  . Setting IAM Policy...                                                      


In [1181]:
!gcloud run services list

   SERVICE            REGION       URL                                                LAST DEPLOYED BY                                     LAST DEPLOYED AT
✔  endpoint-05-fraud  us-central1  https://endpoint-05-fraud-urlxi72dpa-uc.a.run.app  1026793852137-compute@developer.gserviceaccount.com  2022-08-03T23:49:44.801719Z


In [1249]:
services = !gcloud run services list --format="json" --filter=SERVICE:endpoint-$SERIES-$DATANAME
services = json.loads("".join(services))[0]
services['status']['url']

'https://endpoint-05-fraud-urlxi72dpa-uc.a.run.app'

If you had to adjust a `Domain Restricted Sharing` policy after deployment then this command can update the service to allow all traffic:

In [1193]:
#!gcloud run services add-iam-policy-binding --region=us-central1 --member='allUsers' --role=roles/run.invoker endpoint-$SERIES-$DATANAME

### Get Predictions Using Cloud Run Service

In [1250]:
import requests

In [1252]:
headers = {"content-type": "application/json"}
json_response = requests.post(f"{services['status']['url']}/v1/models/fraud:predict", data=json.dumps({"instances": [newobs[0]]}), headers=headers)

In [1253]:
print(json_response.text)

{
    "predictions": [[0.999275744, 0.000724321057]
    ]
}


In [1254]:
predictions = json.loads(json_response.text)['predictions']
predictions

[[0.999275744, 0.000724321057]]

In [1255]:
np.argmax(predictions[0])

0

### Remove Service
Alternatively, you could adjust the service to not accept traffic.  Cloud Run will scale down to zero - or only charge when CPU is used (startup, shutdown, and receiving requests) unless `--no-cpu-throttling` is used ([documentation](https://cloud.google.com/run/docs/configuring/cpu-allocation#setting)).

In [1256]:
!gcloud run services delete --region=us-central1 --quiet endpoint-$SERIES-$DATANAME

Deleting [endpoint-05-fraud]...done.                                           
Deleted service [endpoint-05-fraud].


---